<a href="https://colab.research.google.com/github/idoFinder/NLP_colab/blob/master/CNN_For_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports



In [0]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

from google.colab import drive

In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [0]:
drive.mount("/content/drive")

In [0]:
from google.colab import drive
drive.mount('/content/drive')